# Word Vectors

In [1]:
import spacy

In [2]:
nlp = spacy.load("en_core_web_md")

In [3]:
#A word vector is a multi dimesional

In [4]:
with open("data/wiki_us.txt","r")as f:
    text= f.read()

In [5]:
doc=nlp(text)
sentence1=list(doc.sents)[0]
print(sentence1)
    

The United States of America (U.S.A. or USA), commonly known as the United States (U.S. or US) or America, is a country primarily located in North America.


In [6]:
import numpy as np

your_word = "Country"

ms = nlp.vocab.vectors.most_similar(
    np.asarray([nlp.vocab.vectors[nlp.vocab.strings[your_word]]]), n=10)
words = [nlp.vocab.strings[w] for w in ms[0][0]]
distances = ms[2]
print(words)

['country', 'COUNTRY', 'NATION', 'nation', 'COUNTIRES', 'nations', 'member-states', 'worLd', 'World', 'world']


In [7]:
doc1 = nlp("I like salty fries and hamburgers")
doc2 = nlp("Fast food tastes very good.")

In [8]:
print(doc1,"<->",doc2,doc1.similarity(doc2))


I like salty fries and hamburgers <-> Fast food tastes very good. 0.732822189442892


In [9]:
doc3 = nlp("Empire State building is in newyork")
print(doc1,"<->",doc3,doc1.similarity(doc3))

#spacy is going into it's word embeddings

I like salty fries and hamburgers <-> Empire State building is in newyork 0.42828225973125505


In [10]:
doc4 = nlp("I enjoy oranges")
doc5 = nlp("I enjoy apples")
print(doc4,"<->",doc5,doc4.similarity(doc5))

I enjoy oranges <-> I enjoy apples 0.9445392991167937


In [11]:
# the similarity is not dependent on the contextual words...
#rather its dependent on the semantics of the words

doc6 = nlp("I enjoy burgers.") 
print(doc4,"<->",doc6,doc4.similarity(doc6))


I enjoy oranges <-> I enjoy burgers. 0.8226324017443503


In [12]:
french_fries = doc1[2:4]
burgers= doc1[5]
print(french_fries,"<->",burgers,french_fries.similarity(burgers))

salty fries <-> hamburgers 0.7304624


# Pipelines


A pipeline is a very commmon expression in data science. 
A pipeline is a sequence of pipes or actors on data, that make alterations to the data or extract informarion from it.  

spacy pipeline for ner

In [3]:
nlp = spacy.blank("en")
nlp2= spacy.load("en_core_web_sm")

In [4]:
nlp.add_pipe(nlp.create_pipe('sentencizer'))


In [5]:
import requests
from bs4 import BeautifulSoup
s= requests.get("https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt")
soup = BeautifulSoup(s.content).text.replace("-\n", "").replace("\n", " ")
nlp.max_length = 5278439

In [6]:
%%time
doc = nlp(soup)
print (len(list(doc.sents)))
# this is our own pipeline model which only splits it into senctences 
#if the en_core_web_sm pipeline was used it would have taken 47 minutes to finish

94139
CPU times: user 6.62 s, sys: 60.5 ms, total: 6.68 s
Wall time: 6.68 s


In [8]:
nlp2.analyze_pipes()
#works with spacy 2.0

AttributeError: 'English' object has no attribute 'analyze_pipes'

In [9]:
ruler = nlp.add_pipe("entity_ruler")
ruler.add_patterns(...)

ValueError: [E003] Not a valid pipeline component. Expected callable, but got 'entity_ruler' (name: 'None').[E004] If you meant to add a built-in component, use `create_pipe`: `nlp.add_pipe(nlp.create_pipe('entity_ruler'))`

# Using Spacy's Entity Ruler 

#two ways to create custom spacy pipeline:-
#Rules Based approach
#Machine learning Based Approach

Good example for when to use machine lerning and when to use reg ex or rules:

when there are a finite number of forms say date....only finite possible outcomes

Machine learning can be used for names cause names are different 

In [22]:
nlp = spacy.load("en_core_web_sm")
text = "West Chestertenfieldville was referenced in Mr. Deeds."

In [21]:
doc  = nlp(text)

In [20]:
for ent in doc.ents:
    print(ent.text,ent.label_)

em_core_web_sm model is a ml model for ner , but even a very robust machine learning model would identify west chestertenfield as a person 

In [23]:
ruler = nlp.add_pipe(nlp.create_pipe("entity_ruler"))
#adds entity ruler to last pipe of en_core_web_sm

In [24]:
patterns = [{"label":"GPE" , "pattern":"West Chestertenfieldville"}]
ruler.add_patterns(nlp.create_patterns(patterns))

AttributeError: 'NoneType' object has no attribute 'add_patterns'

In [ ]:
import spacy
import re
from spacy.language import Language

boundary = re.compile('^[0-9]$')

@Language.component("component")
def custom_seg(doc):
    prev = doc[0].text
    length = len(doc)
    for index, token in enumerate(doc):
        if (token.text == '.' and boundary.match(prev) and index!=(length - 1)):
            doc[index+1].sent_start = False
        prev = token.text
    return doc
    
nlp.add_pipe("component", before='parser')